In [5]:
# Importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
url = 'https://raw.githubusercontent.com/EmmanuelNDAHIMANA/Hamoye_Assignment/master/energydata_complete.csv'
energydata = pd.read_csv(url)

In [4]:
#Rename the columns for better identification
column_namess = { 'Appliances':'Target', 'T1':'Temp_kitc', 'RH_1':'Hum_kitc', 'T2':'Temp_living_room',
                 'RH_2':'Hum_living_room', 'T3':'Temp_laundry','RH_3':'Hum_laundry', 'T4':'Temp_office',
                 'RH_4':'Hum_office', 'T5':'Temp_bathroom', 'RH_5':'Hum_bathroom', 'T6':'Temp_outside',
                 'RH_6':'Hum_outside', 'T7':'Temp_ironing_room', 'RH_7':'Hum_ironing_room', 
                 'T8':'Temp_teenager_room','RH_8':'Hum_teenager_room', 'T9':'Temp_parents_room', 
                 'RH_9':'Hum_parents_room', 'T_out':'Temp_outside_CWS', 'Press_mm_hg':'Pressure_CWS',
                 'RH_out':'Hup_outside_CWS', 'Windspeed':'Wind_speed_CWS','Visibility':'Visibility_CWS',
                 'Tdewpoint':'Tdewpoint_CWS', 'rv1':'Random_var1', 'rv2':'Random_var2'
} 
energydata = energydata.rename(columns=column_namess)

In [6]:
# Dropping some columns before we scal the data set
energydata = energydata.drop(columns = ['date', 'lights'])

In [7]:
# Scalling the data set before splitting them and define the predictors and target data as well.
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler

minmax = MinMaxScaler()

energydata_scalled = pd.DataFrame(minmax.fit_transform(energydata), columns=energydata.columns)
energydata_predictors = energydata_scalled.drop(columns = ['Target'])
energydata_target = energydata_scalled[['Target']]

In [8]:
#Split the data into training and testing set with 30% of test with 42% random state.
x_train, x_test, y_train, y_test = train_test_split(energydata_predictors, energydata_target, test_size = 0.3, random_state = 42)

In [11]:
# Fitting the multiple linear regression with grid search for optimal parametors
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

lin_mod = LinearRegression()
lin_model = lin_mod.fit(x_train, y_train)
pred_lin = lin_model.predict(x_test)

In [12]:
#From the dataset, fit a linear model on the relationship between the temperature
#in the living room in Celsius (x = T2) and the temperature outside the building (y = T6).
#What is the R^2 value in two D.P?
linear_model = LinearRegression().fit(energydata[['Temp_living_room']], energydata[['Temp_outside']])
predicted_values = linear_model.predict(energydata[['Temp_living_room']])
print("R-square:",round(r2_score(energydata[['Temp_outside']], predicted_values), 2))

R-square: 0.64


In [13]:
# What is the Mean Absolute Error (in two decimal places)?
print("MAE:", round(mean_absolute_error(energydata[['Temp_outside']], predicted_values),2))

MAE: 2.84


In [14]:
# What is the Residual Sum of Squares (in two decimal places)
print("Residual sum of squares: %.2f" % ((predicted_values - energydata[['Temp_outside']]) ** 2).sum())

Residual sum of squares: 262122.67


In [15]:
# What is the Root Mean Squared Error (in three decimal places)?
print("MSE:", round(np.sqrt(mean_squared_error(energydata[['Temp_outside']], predicted_values)),3))

MSE: 3.644


In [16]:
# What is the Coefficient of Determination (in two decimal places)? 
print("R-square:",round(r2_score(energydata[['Temp_outside']], predicted_values), 3))

R-square: 0.642


In [17]:
# Obtain the feature weights from your linear model above. 
# Which features have the lowest and highest weights respectively?

for coef, col in enumerate(x_train.columns):
    print(f'{col}:  {lin_model.coef_[coef]}')

Temp_kitc:  [-0.00328105  0.5535466  -0.23617792 -0.45669795  0.29062714  0.09604827
  0.028981    0.02638578 -0.01565684  0.01600579  0.23642491  0.03804865
  0.01031878 -0.04461364  0.10199505 -0.15759548 -0.18994077 -0.03980032
 -0.32185967  0.00683933 -0.07767065  0.02918313  0.01230661  0.11775773
  0.0007701   0.0007701 ]


IndexError: index 1 is out of bounds for axis 0 with size 1

In [18]:
# Train a ridge regression model with an alpha value of 0.4. Is there any 
# change to the root mean squared error (RMSE)
# when evaluated on the test set?
ridge = Ridge(alpha=0.4).fit(x_train, y_train)

ridge_p = ridge.predict(x_test)
print("MSE:", round(np.sqrt(mean_squared_error(y_test, ridge_p)),3))

MSE: 0.088


In [20]:
# What is the new RMSE with the Lasso Regression (in 3 decimal places)?
lass_mod = Lasso()
lass_model = lass_mod.fit(x_train, y_train)
pred_lass  = lass_model.predict(x_test)
print("MSE:", round(np.sqrt(mean_squared_error(y_test, pred_lass)),3))

MSE: 0.095
